In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
paths=[]
labels=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label=filename.split('_')[-1]
        label=label.split('.')[0]
        labels.append(label.lower())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'seaborn'

In [3]:
paths[:5]

NameError: name 'paths' is not defined

In [20]:
labels[:5]

NameError: name 'labels' is not defined

In [21]:
df=pd.DataFrame()
df['speech']=paths
df['label']=labels
df.head()

NameError: name 'paths' is not defined

In [22]:
df['label'].value_counts()

KeyError: 'label'

In [23]:
def waveplot(data,sr,emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion,size=20)
    librosa.display.waveshow(data,sr=sr)
    plt.show()


def spectogram(data, sr, emotion):
    x=librosa.stft(data)
    xdb=librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(10, 4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()
    plt.show()
    

In [24]:
emotion='fear'
path=np.array(df['speech'][df['label']==emotion])[1]
data,sampling_rate=librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectogram(data,sampling_rate,emotion)
Audio(path)

KeyError: 'speech'

In [ ]:
emotion='neutral'
path=np.array(df['speech'][df['label']==emotion])[1]
data,sampling_rate=librosa.load(path)
waveplot(data,sampling_rate,emotion)
spectogram(data,sampling_rate,emotion)
Audio(path)

# Feature Extraction

In [25]:
def extract_mfcc(filename):
    y,sr=librosa.load(filename,duration=3,offset=0.5)
    mfcc=np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)
    return mfcc

In [26]:
extract_mfcc(df['speech'][0])

KeyError: 'speech'

In [ ]:
X_mfcc=df['speech'].apply(lambda x:extract_mfcc(x))

In [ ]:
X_mfcc


In [27]:
X=[x for x in X_mfcc]
X=np.array(X)
X.shape

NameError: name 'X_mfcc' is not defined

In [28]:
X=np.expand_dims(X,-1)
X.shape

NameError: name 'X' is not defined

In [29]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
y=enc.fit_transform(df[['label']])

KeyError: "None of [Index(['label'], dtype='object')] are in the [columns]"

In [ ]:
y=y.toarray()

In [ ]:
y.shape

In [30]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

model=Sequential([
    
    LSTM(123,return_sequences=False,input_shape=(40,1)),
    Dense(64,activation='relu'),
    Dropout(0,2),
    Dense(32,activation='relu'),
    Dropout(0,2),
    Dense(7,activation='softmax')
    
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

ModuleNotFoundError: No module named 'keras'

In [ ]:
history=model.fit(X,y,validation_split=0.2,epochs=70,batch_size=512,shuffle=True)

In [31]:
epochs=list(range(70))
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
plt.plot(epochs,acc,label="train_accuracy")
plt.plot(epochs,val_acc,label="val_accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.save('your_model_name.h5')
